In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression as LR

In [2]:
root_dir = '/Users/kessapassa/OneDrive/research_log/2018_Graduate/3D/'

In [3]:
dir_list = ['people10000', 'people20000', 'people30000']
seed_list = [str(123 + i) for i in range(3)]
csv_list = ['census', 'mobile']

In [4]:
loc_list = [14, 15, 21, 22, 27, 28]

In [5]:
csv_array = {}
for _dir in dir_list:
    csv_array[_dir] = {}
    for _seed in seed_list:
        csv_array[_dir][_seed] = {}
        for _csv in csv_list:
            df = pd.read_csv(root_dir + _dir + 'seed' + _seed + '_' + _csv + '.csv',
                                                index_col=0,
                                                encoding='Shift_JISx0213')
            csv_array[_dir][_seed][_csv] = df

In [6]:
# なぜかこの時だけ21600がないので0で補間する
csv_array['people10000']['123']['census']['21600'] = 0

In [7]:
def get_ratio_per_sum(base):
    for index in range(len(base.columns)):
        base.iloc[:, index] /= base.iloc[:, index].sum()
#     base = base.round(4)
    
    return base

In [8]:
# for _dir in dir_list:
#     for _seed in seed_list:
#         for _csv in csv_list:
#             df = csv_array[_dir][_seed][_csv].copy()
#             df = get_ratio_per_sum(df)
#             df = df.fillna(0)
#             csv_array[_dir][_seed][_csv] = df

# 1時間後にどこに移動するか割合を推定する

In [9]:
times_list = [str(3600 * (i + 1)) for i in range(6)]

In [10]:
def get_model(trainX, y):
    model = LR()
    model.fit(trainX, y)
    
    return model

In [11]:
def interpolate_times(index):
    return times_list[:index+1]

In [12]:
pred = None

pred_array = {}
for _dir in dir_list:
    pred_array[_dir] = {}
    for _seed in seed_list:
        pred_array[_dir][_seed] = {}
        for _csv in csv_list:
            pred_array[_dir][_seed][_csv] = {}
            
            for index in range(len(times_list)-1):
                # seed123だけを学習し、他はテストとする
                df = csv_array[_dir][_seed][_csv].copy()
                df = get_ratio_per_sum(df)
                df = df.fillna(0)
                
                if _seed == '123':
                    model = get_model(df[interpolate_times(index)], df[times_list[index+1]])
                    pred = model.predict(df[interpolate_times(index)])
                    
                else:
                    df = df[interpolate_times(index+1)]
                    df['pred'] = pred
                    
                    columns = df.columns
                    df['diff'] = df[columns[-2]] / df[columns[-1]]
                    pred_array[_dir][_seed][_csv][times_list[index+1]] = df

In [13]:
def get_times_list(time):
    index = times_list.index(time)
    return times_list[:index+1]

In [14]:
people = 'people20000'
seed = '125'
csv = 'census'
time = '10800'

In [15]:
locked_times_list = get_times_list(time)
df = csv_array[people][seed][csv][locked_times_list].copy()
df['pred'] = pred_array[people][seed][csv][time]['pred'] * csv_array[people][seed][csv][time].sum()
df['diff'] = df[time] / df['pred']
df.loc[loc_list, :]

,3600,7200,10800,pred,diff
14,2123.0,175.0,72.0,28.797239,2.500240
15,1504.0,117.0,114.0,23.218707,4.909834
21,4717.0,874.0,812.0,1022.702781,0.793975
22,2457.0,620.0,569.0,656.624444,0.866553
27,5002.0,673.0,870.0,1202.745766,0.723345
28,3697.0,1182.0,752.0,519.805946,1.446694


In [16]:
df['diff'].describe()

count     36.000000
mean       6.219147
std       19.484643
min      -10.120656
25%        0.000000
50%        0.758660
75%        2.951428
max      109.186271
Name: diff, dtype: float64

In [17]:
csv_array[people][seed][csv] / csv_array[people][seed][csv].sum()

,3600,7200,10800,14400,18000,21600
0,0.000610,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000648,0.000000,0.000000,0.000766,0.000000,0.001453
2,0.001525,0.000874,0.000279,0.000383,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.032521,0.015072,0.008373,0.006128,0.008480,0.005087
7,0.001029,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.012315,0.004369,0.002512,0.003447,0.001305,0.002907
9,0.004918,0.001311,0.000558,0.000766,0.000000,0.000000
